## Анализ романов "Пейзаж, нарисованный чаем" и "Биография Белграда" с помощью цифровых методов.
#### Два основных раздела: ключевые слова и извлечение именованных сущностей. 

Ссылка на презентацию: 

Импортируем всё, что потребуется.

In [1]:
import json, os
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
morph = MorphAnalyzer()
stops = set(stopwords.words('russian'))
import re

from natasha import LocationExtractor, NamesExtractor
from bs4 import BeautifulSoup
import pandas as pd
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/laidhimonthegreen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from string import punctuation
from nltk.corpus import stopwords
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

def normalize(bag):
    
    words = [word.strip(punct) for word in bag]
    words = [morph.parse(word)[0] for word in words if word and word not in stops]
    words = [word.normal_form for word in words if word.tag.POS == 'NOUN']

    return words

## Ключевые слова

### Находим самые частотные слова для "Пейзажа"

In [3]:
def file_to_words(path):
    words = []
    with open(path, "r") as f:
        for line in f.readlines():
            if line.strip():  
                words += line.strip().lower().split()   
    return words

In [4]:
file_to_words("pavich.txt")

['милорад',
 'павич',
 'пейзаж,',
 'нарисованный',
 'чаем',
 'роман',
 'перевод',
 'с',
 'сербского',
 'н.',
 'вагаповой',
 'и',
 'р.',
 'грецкой.',
 'книга',
 'первая.',
 'маленький',
 'ночной',
 'роман',
 'оглавление',
 '1.',
 'концы',
 'усов',
 'у',
 'них',
 'висели',
 'как',
 'плети',
 '(кругом',
 'одно',
 'горе,',
 'и',
 'все',
 'мы',
 'в',
 'нем',
 'точно',
 'рыба',
 'в',
 'воде.)',
 '2.',
 'первые',
 'века',
 'они',
 'жили',
 'в',
 'синайской',
 'пустыне',
 '(ударь',
 'палкой',
 'по',
 'кусту',
 '—',
 'вырастет',
 'цветок.)',
 '3.',
 'однажды',
 'ночью',
 'императрице',
 'теодоре',
 'приснилось,',
 'что',
 'к',
 'ней',
 'в',
 'опочивальню',
 'слетаются',
 'ангелы',
 '(громы',
 'и',
 'молнии',
 'сплошь',
 'исхлестали',
 'и',
 'землю,',
 'и',
 'воду.)',
 '4.',
 'неизвестно,',
 'какой',
 'из',
 'двух',
 'монашеских',
 'укладов',
 'предпочитал',
 'греческий,',
 'дославянский',
 'хилендар',
 '(эгейское',
 'море',
 'спокойно',
 'только',
 'по',
 'воскресным',
 'дням',
 'и',
 'по',
 'п

In [5]:
norm_words = normalize(file_to_words("pavich.txt"))

In [20]:
a = Counter(norm_words).most_common(50)

In [21]:
import random
d = []
for i in a:
    for n in range(1, i[1]):
        d.append(i[0])
    
random.shuffle(d)

In [7]:
norm_words_2 = normalize(file_to_words("part_two.txt"))

In [8]:
Counter(norm_words_2).most_common(20)

[('витач', 233),
 ('разин', 213),
 ('время', 203),
 ('человек', 179),
 ('год', 154),
 ('глаз', 130),
 ('день', 123),
 ('жизнь', 113),
 ('дом', 108),
 ('рука', 105),
 ('дело', 105),
 ('книга', 103),
 ('голос', 102),
 ('архитектор', 98),
 ('вид', 94),
 ('госпожа', 93),
 ('слово', 90),
 ('афанасий', 85),
 ('сон', 81),
 ('амалия', 78)]

In [9]:
d = [x[0] for x in Counter(norm_words_2).most_common(20)]

In [10]:
print (d)

['витач', 'разин', 'время', 'человек', 'год', 'глаз', 'день', 'жизнь', 'дом', 'рука', 'дело', 'книга', 'голос', 'архитектор', 'вид', 'госпожа', 'слово', 'афанасий', 'сон', 'амалия']


#### Разобьём повествование в первой части на главы про XX век и на главы-легенды

In [11]:
landscape = ""

with open("svilar_annotated.txt", "r+") as f:
    for line in f:
        if line.strip():
            landscape += line

In [12]:
landscape

'1\n<legend> \nКонцы усов у них висели как плети. Поколение за поколением жили они без единой улыбки, и годы помечали морщинами лишь верхнюю часть их лиц, и старели они не от удовольствий, а от мыслей.\nГоворят, иудеи прозвали их эдомеями. Сами же себя они называли — «соль». Много времени пройдет, пока человек съест горстку соли, — вот что они имели в виду. Эдомеи отличались терпением. У них было два знака — знак Агнца и знак Рыбы. Агнцу посвящались пироги, замешенные на слезах, а Рыбе — обручальные кольца из теста (Рыба — невеста души).\nРазделение это случилось не вдруг. Четыре или пять колен прошло, пока один из них сказал: «Ничего нет на свете лучше говорящего дерева. Ведь дерево дает плоды обоего пола. По ним можно отличить тишину от молчания. Ибо человек, чье сердце полно молчания, совсем не таков, как тот, чье сердце исполнено тишины…»\nАнтиохиец, сказавший эту фразу, едва успев ее произнести, без страха и ненависти принял смерть от зубов хищного зверя, как и его соплеменник Игн

In [13]:
soup = BeautifulSoup(landscape, 'html.parser')
aphanas = str(soup.find_all("aphanas"))
aphanas_texts = aphanas.split("<aphanas>")[1:]

In [14]:
def text_to_keys(text):
    bag = text.lower().split()
    words = normalize(bag)
    return Counter(words).most_common(20)    

In [15]:
def text_to_str(text):
    bag = text.lower().split()
    words = normalize(bag)
    str_words = " ".join(words)
    return str_words

In [16]:
text_to_keys(aphanas)

[('свилара', 145),
 ('год', 71),
 ('время', 62),
 ('человек', 55),
 ('отец', 53),
 ('афанасий', 47),
 ('день', 44),
 ('язык', 44),
 ('жизнь', 43),
 ('глаз', 42),
 ('монастырь', 40),
 ('сын', 37),
 ('рука', 37),
 ('вода', 36),
 ('вино', 35),
 ('волос', 33),
 ('монах', 32),
 ('слово', 31),
 ('дом', 31),
 ('ночь', 29)]

In [17]:
legend = str(soup.find_all("legend"))
legend_texts = legend.split("<legend>")[1:]

In [18]:
text_to_keys(legend)

[('монастырь', 41),
 ('общинник', 37),
 ('монах', 33),
 ('одиночка', 26),
 ('икона', 22),
 ('время', 21),
 ('имя', 19),
 ('жизнь', 18),
 ('язык', 16),
 ('карамустафа', 16),
 ('хилендар', 16),
 ('церковь', 15),
 ('сын', 15),
 ('гора', 14),
 ('грек', 14),
 ('уклад', 14),
 ('богородица', 13),
 ('день', 12),
 ('друг', 11),
 ('савва', 11)]

Сделаем таблицу, чтобы хранить в ней информацию по нашим ключевым словам. 

In [19]:
aphanas_df = pd.DataFrame({
        'text': aphanas_texts,
        'tag': "aphanas"
 })

In [32]:
#удаляем стоп-слова, оставляем существительные
def text_to_bag(text):
    bagwords = text.lower().split()
    normwords = normalize(bagwords)
    return normwords

In [61]:
aphanas_df["nouns"] = aphanas_df["text"].apply(text_to_bag)

In [62]:
legend_df = pd.DataFrame({
        'text': legend_texts,
        'tag': "legend"
 })

In [63]:
legend_df["nouns"] = legend_df["text"].apply(text_to_bag)

In [64]:
landscape_df = pd.concat([aphanas_df, legend_df])

In [65]:
landscape_df

,text,tag,nouns
0,"\nВедь каждый идиоритмик молчит сам по себе, а...",aphanas,"[идиоритмика, тишина, человек, молчание, пол, ..."
1,\nОбыкновенно в сумерки они приходили в полура...,aphanas,"[сумерки, дом, врачар, дождь, река, дунай, сав..."
2,\nНа улице недалеко от кладбища Святого Никола...,aphanas,"[улица, кладбище, николай, маленькая, чаща, вы..."
3,"\nВсе тогда набросились на языки, как недавно ...",aphanas,"[язык, дар, речь, санскрит, речь, жизнь, смерт..."
4,\nУ этой красавицы улыбка была такая неглубока...,aphanas,"[красавица, улыбка, собеседник, нос, мель, поц..."
5,\nВ ту осень исполнялась еще одна седмица его ...,aphanas,"[осень, седмица, год, год, профессия, труд, го..."
6,\nОн клевал носом над тарелкой молочного супа ...,aphanas,"[нос, тарелка, суп, укроп, ложка, пара, сын, в..."
7,\nАфанасий Свилар расставил на плане Белграда ...,aphanas,"[афанасий, свилара, план, белград, пометка, ди..."
8,"\nБыл один из тех дней, когда сбывается послов...",aphanas,"[день, пословица, палка, куст, цветок, день, а..."
9,"\nПредполагая, что майор Коста Свилар попал из...",aphanas,"[майор, коста, свилара, албания, греция, море,..."


### Находим ключевые слова для "Пейзажа" по tf-idf

In [39]:
def strize(lst):
    stroka = " ".join(lst)
    return stroka

In [67]:
landscape_df["nouns_to_str"] = landscape_df["nouns"].apply(strize)

In [68]:
landscape_df["sentences"] = landscape_df["text"].apply(sent_tokenize)

Создадим корпус текстов на основе строк из существительных без стоп-слов.

In [69]:
corpus = []
for i, line in enumerate(landscape_df["nouns_to_str"]):
    corpus.append(landscape_df["nouns_to_str"].iloc[i])

Переходим к использованию tf-idf.

In [70]:
tfidf = TfidfVectorizer(ngram_range=(1,1))
X_corpus = tfidf.fit_transform(corpus)
X_legend_vs_XX = tfidf.fit_transform([aphanas, legend])

In [71]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors_corpus = tfidf.transform(corpus)
keywords_corpus = [[id2word[w] for w in top] for top in texts_vectors_corpus.toarray().argsort()[:,:-20:-1]] 

Ключевые слова для каждой из глав.

In [72]:
keywords_corpus

[['свилара',
  'опсеница',
  'проект',
  'человек',
  'год',
  'улица',
  'неприязнь',
  'работа',
  'афанасий',
  'рука',
  'лицо',
  'город',
  'время',
  'место',
  'жизнь',
  'ночь',
  'колодец',
  'воспоминание',
  'лихорадка'],
 ['книга',
  'полка',
  'стакан',
  'птица',
  'этаж',
  'вино',
  'волос',
  'ответ',
  'ум',
  'зеркало',
  'карман',
  'целое',
  'кончик',
  'коса',
  'кран',
  'жидкость',
  'дверь',
  'улица',
  'сумерки'],
 ['лото',
  'вино',
  'зал',
  'игра',
  'ветер',
  'затылок',
  'улица',
  'осень',
  'рыба',
  'нога',
  'стол',
  'сон',
  'горло',
  'листок',
  'заря',
  'афанасий',
  'стерлядь',
  'обыкновение',
  'река'],
 ['язык',
  'слово',
  'друг',
  'урок',
  'волос',
  'время',
  'память',
  'снег',
  'губа',
  'день',
  'сабля',
  'улица',
  'занятие',
  'небо',
  'учитель',
  'война',
  'афанасий',
  'собака',
  'рука'],
 ['лодка',
  'щенок',
  'газета',
  'ветер',
  'волос',
  'дым',
  'мель',
  'дно',
  'собеседник',
  'горечь',
  'сабля',
  'вол

In [73]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors_legaph = tfidf.transform([text_to_str(aphanas), text_to_str(legend)])
keywords_legaph = [[id2word[w] for w in top] for top in texts_vectors_legaph.toarray().argsort()[:,:-20:-1]] 

Ключевые слова отдельно для XX века и легенд.

In [76]:
print (keywords_legaph[0])

['свилара', 'год', 'афанасий', 'время', 'человек', 'отец', 'рука', 'язык', 'день', 'жизнь', 'глаз', 'монастырь', 'книга', 'война', 'сын', 'вода', 'земля', 'вино', 'монах']


In [59]:
landscape_df["tf-idf"] = keywords_corpus

Взглянем на ключевые слова для каждой из глав.

In [62]:
for i, words in enumerate(landscape_df["tf-idf"]):
    print (landscape_df["sentences"].iloc[i][0])
    print (words)


Ведь каждый идиоритмик молчит сам по себе, а кенобиты хранят свою общую тишину.
['свилара', 'опсеница', 'проект', 'человек', 'год', 'улица', 'неприязнь', 'работа', 'афанасий', 'рука', 'лицо', 'город', 'время', 'место', 'жизнь', 'ночь', 'колодец', 'воспоминание', 'лихорадка']

Обыкновенно в сумерки они приходили в полуразрушенный дом на Врачаре, от которого дожди всегда стекали в две реки сразу: в Дунай и в Саву.
['книга', 'полка', 'стакан', 'птица', 'этаж', 'вино', 'волос', 'ответ', 'ум', 'зеркало', 'карман', 'целое', 'кончик', 'коса', 'кран', 'жидкость', 'дверь', 'улица', 'сумерки']

На улице недалеко от кладбища Святого Николая, в маленькой закусочной, чаще сменявшей вывески, чем клиентов, которые называли ее по привычке «В кредит», у него прорезались первые усы.
['лото', 'вино', 'зал', 'игра', 'ветер', 'затылок', 'улица', 'осень', 'рыба', 'нога', 'стол', 'сон', 'горло', 'листок', 'заря', 'афанасий', 'стерлядь', 'обыкновение', 'река']

Все тогда набросились на языки, как недавно обр

### Находим самые частотные слова для "Белграда"

In [7]:
bel_norm_words = normalize(file_to_words("belgrad.txt"))

In [18]:
a = Counter(bel_norm_words).most_common(25)

In [19]:
print (a)

[('белград', 251), ('год', 132), ('город', 115), ('время', 59), ('век', 52), ('сербия', 49), ('турок', 35), ('дунай', 29), ('мир', 28), ('река', 27), ('поэт', 27), ('место', 26), ('сторона', 25), ('сава', 25), ('день', 23), ('король', 22), ('крепость', 21), ('церковь', 21), ('власть', 20), ('книга', 20), ('карагеоргий', 20), ('улица', 19), ('писатель', 19), ('здание', 19), ('башня', 18)]


In [25]:
belgrad_str = ""
with open("belgrad.txt", "r+") as f:
    for line in f:
        if line.strip():
            belgrad_str += line  

In [26]:
from string import digits

In [27]:
belgrad_texts = belgrad_str.split("<part")[1:]
belgrad_texts = [text.lstrip("=\"").lstrip(digits).lstrip("\">") for text in belgrad_texts]

In [28]:
belgrad_df = pd.DataFrame({
        'text': belgrad_texts, 
 })

In [29]:
belgrad_df["sentences"] = belgrad_df["text"].apply(sent_tokenize)

In [30]:
belgrad_df

,text,sentences
0,"\n1\nКто хочет овладеть крепостью, тому придет...","[\n1\nКто хочет овладеть крепостью, тому приде..."
1,\n2\nВ Сингидунуме обнаружены и следы раннего ...,[\n2\nВ Сингидунуме обнаружены и следы раннего...
2,"\n3\nВ следующем, XV веке Белград восстановил ...","[\n3\nВ следующем, XV веке Белград восстановил..."
3,\n4\nПодобно тому как в XI и XII веках через Б...,[\n4\nПодобно тому как в XI и XII веках через ...
4,\n5\nВ Средневековье и позже часто упоминается...,[\n5\nВ Средневековье и позже часто упоминаетс...
5,\n6\nВ Белграде XIX века кипела литературная и...,[\n6\nВ Белграде XIX века кипела литературная ...
6,\n7\nНовое разрушение Белград претерпел от рук...,[\n7\nНовое разрушение Белград претерпел от ру...
7,\n8\nВ 1948 году Иосип Броз Тито обнародовал р...,[\n8\nВ 1948 году Иосип Броз Тито обнародовал ...
8,\n9\nТретье тысячелетие началось в Белграде на...,[\n9\nТретье тысячелетие началось в Белграде н...
9,\n10\nНо некоторые районы Белграда поистине во...,[\n10\nНо некоторые районы Белграда поистине в...


In [33]:
belgrad_df["nouns"] = belgrad_df["text"].apply(text_to_bag)

In [34]:
belgrad_df

,text,sentences,nouns
0,"\n1\nКто хочет овладеть крепостью, тому придет...","[\n1\nКто хочет овладеть крепостью, тому приде...","[крепость, душа, путь, белград, город, мир, ча..."
1,\n2\nВ Сингидунуме обнаружены и следы раннего ...,[\n2\nВ Сингидунуме обнаружены и следы раннего...,"[сингидунум, след, христианство, время, диокле..."
2,"\n3\nВ следующем, XV веке Белград восстановил ...","[\n3\nВ следующем, XV веке Белград восстановил...","[век, белград, столица, город, властитель, дес..."
3,\n4\nПодобно тому как в XI и XII веках через Б...,[\n4\nПодобно тому как в XI и XII веках через ...,"[век, белград, палестина, участник, поход, век..."
4,\n5\nВ Средневековье и позже часто упоминается...,[\n5\nВ Средневековье и позже часто упоминаетс...,"[средневековье, башня, белград, фольклор, имя,..."
5,\n6\nВ Белграде XIX века кипела литературная и...,[\n6\nВ Белграде XIX века кипела литературная ...,"[белград, век, жизнь, участник, министр, просв..."
6,\n7\nНовое разрушение Белград претерпел от рук...,[\n7\nНовое разрушение Белград претерпел от ру...,"[новое, разрушение, белград, рука, австро-венг..."
7,\n8\nВ 1948 году Иосип Броз Тито обнародовал р...,[\n8\nВ 1948 году Иосип Броз Тито обнародовал ...,"[год, иосип, броза, тито, решение, отделение, ..."
8,\n9\nТретье тысячелетие началось в Белграде на...,[\n9\nТретье тысячелетие началось в Белграде н...,"[третье, тысячелетие, белград, год, год, стран..."
9,\n10\nНо некоторые районы Белграда поистине во...,[\n10\nНо некоторые районы Белграда поистине в...,"[район, белград, история, страница, порядок, п..."


In [35]:
def common_20(words):
    return [x[0] for x in Counter(words).most_common(20)]

In [36]:
belgrad_df["common_words"] = belgrad_df["nouns"].apply(common_20)

In [40]:
belgrad_df["nouns_to_str"] = belgrad_df["nouns"].apply(strize)

Посмотрим на самые частые слова для каждой из глав.

In [41]:
for i, words in enumerate(belgrad_df["common_words"]):
    print (belgrad_df["sentences"].iloc[i][0])
    print (words)


1
Кто хочет овладеть крепостью, тому придется сначала овладеть собственной душой.
['белград', 'город', 'имя', 'река', 'дунай', 'век', 'сингидунум', 'купальня', 'место', 'вод', 'время', 'поселение', 'год', 'поэт', 'крепость', 'душа', 'путь', 'мир', 'часть', 'легенда']

2
В Сингидунуме обнаружены и следы раннего христианства.
['белград', 'год', 'король', 'город', 'век', 'время', 'император', 'сингидунум', 'имя', 'королевство', 'река', 'сербия', 'путь', 'камень', 'берег', 'сава', 'правило', 'королева', 'христиана', 'мученик']

3
В следующем, XV веке Белград восстановил и отстроил, превратив его в столицу и великолепный торговый город, другой известный сербский властитель, деспот Стефан Лазаревич; он был представителем европейского гуманизма, притом византийского, более древнего, чем западный.
['белград', 'город', 'год', 'деспот', 'турок', 'возвышение', 'христианство', 'стефан', 'церковь', 'дунай', 'башня', 'запад', 'лазарь', 'место', 'богородица', 'река', 'царьград', 'век', 'поэт', 'слов

### Находим ключевые слова для "Белграда" по tf-idf

In [42]:
bel_corpus = []
for i, line in enumerate(belgrad_df["nouns_to_str"]):
    bel_corpus.append(belgrad_df["nouns_to_str"].iloc[i])

In [43]:
tfidf = TfidfVectorizer(ngram_range=(1,1))

In [44]:
X_bel = tfidf.fit_transform(bel_corpus)

In [45]:
id2word = {i:word for i,word in enumerate(tfidf.get_feature_names())}
texts_vectors_bel = tfidf.transform(bel_corpus)
bel_keywords = [[id2word[w] for w in top] for top in texts_vectors_bel.toarray().argsort()[:,:-20:-1]] 

In [46]:
belgrad_df["tf-idf"] = bel_keywords

In [47]:
for i, words in enumerate(belgrad_df["tf-idf"]):
    print (belgrad_df["sentences"].iloc[i][0])
    print (words)


1
Кто хочет овладеть крепостью, тому придется сначала овладеть собственной душой.
['купальня', 'сингидунум', 'имя', 'поселение', 'белград', 'река', 'вод', 'город', 'дунай', 'век', 'эра', 'аргонавт', 'неолит', 'племя', 'место', 'душа', 'территория', 'лагерь', 'год']

2
В Сингидунуме обнаружены и следы раннего христианства.
['король', 'белград', 'год', 'сингидунум', 'город', 'император', 'век', 'правило', 'королевство', 'время', 'королева', 'имя', 'прибытие', 'симонида', 'почесть', 'радость', 'кателина', 'архиепископ', 'византиец']

3
В следующем, XV веке Белград восстановил и отстроил, превратив его в столицу и великолепный торговый город, другой известный сербский властитель, деспот Стефан Лазаревич; он был представителем европейского гуманизма, притом византийского, более древнего, чем западный.
['белград', 'деспот', 'город', 'турок', 'возвышение', 'год', 'христианство', 'запад', 'стефан', 'поэма', 'ариосто', 'иерусалим', 'богородица', 'царьград', 'лазарь', 'башня', 'константин', 'ос

## Исторические личности в произведениях Павича

### На материале "Биографии Белграда".

In [48]:
get_names = NamesExtractor()

In [49]:
def name_getter(text):
    names = [] 
    matches_names = get_names(text)
    for match in matches_names:
        names.append (text[int(match.span[0]):int(match.span[1])]) 
    return (names)

In [50]:
def all_names(column, getter):
    all_names = []
    for text in column:
        all_names += getter(text)
    return [morph.parse(word)[0].normal_form for word in all_names]

In [51]:
belgrad_names = all_names(belgrad_df["text"], name_getter)

In [52]:
belgrad_names

['адриатик',
 'саву западный',
 'дунай',
 'сингидунум',
 'аполлон родосский',
 'калемегдан',
 'сингидунум',
 'сингидунум',
 'элий тэта',
 'вергилий',
 'катулл',
 'сингидунум',
 'тиберий',
 'септимия',
 'север',
 'диоклетиан',
 'сингидунум',
 'сингидунум',
 'диоклетиан',
 'сингидунум',
 'хермила',
 'аурелия виктор',
 'иовиан',
 'верония',
 'сингидунум',
 'сингидунум',
 'секундиана',
 'юстиниан',
 'атилла',
 'сингидунум',
 'флавий',
 'иоанн',
 'кирилл',
 'мефодий',
 'моравия',
 'климент',
 'константин',
 'людовик',
 'фридрих',
 'барбаросс',
 'йованка',
 'калич мийушк',
 'неманич',
 'драгутиный',
 'драгутиный',
 'данте',
 'данило',
 'симонида',
 'милутиный',
 'андроник',
 'симонида',
 'кателина',
 'данило',
 'стефан',
 'кателина',
 'стефан',
 'стефан лазаре',
 'стефан лазаре',
 'стефан лазаре',
 'николай',
 'святой параскева',
 'сиона',
 'давидов',
 'стефан',
 'константин философ',
 'бертрандон',
 'святого николай',
 'джурадж бранк',
 'сигизмунд',
 'джурадж',
 'фурланий',
 'кантакузина',


In [211]:
Counter(belgrad_names).most_common(10)

[('леандр', 12),
 ('сингидунум', 10),
 ('александр', 6),
 ('стефан', 5),
 ('йован', 5),
 ('каменский', 5),
 ('карагеоргий', 5),
 ('андерсен', 5),
 ('калемегдан', 4),
 ('небойша', 4)]

К сожалению, natasha не слишком качественно работает в случаях, когда имена не являются типично русскими.

### На материале "Пейзажа, нарисованного чаем".

In [214]:
landscape_names = all_names(landscape_df["text"], name_getter)

In [215]:
Counter(landscape_names).most_common(10)

[('свилара', 141),
 ('афанасий', 45),
 ('карамустафа', 17),
 ('коста', 13),
 ('василий', 12),
 ('савва', 12),
 ('йоан', 7),
 ('йован', 6),
 ('степанида', 6),
 ('неманя', 5)]

## Локации в произведениях Павича

In [216]:
get_location = LocationExtractor()

In [217]:
def location_getter(text):
    locations = [] 
    matches_locations = get_location(text)
    for match in matches_locations:
        locations.append (text[int(match.span[0]):int(match.span[1])]) 
    return (locations)

In [225]:
locations = all_names([landscape], location_getter)

In [226]:
Counter(locations).most_common(10)

[('тот', 44),
 ('белград', 18),
 ('карамустафа', 18),
 ('куда', 16),
 ('афон', 16),
 ('нея', 12),
 ('греция', 12),
 ('никола', 11),
 ('красный', 9),
 ('который', 8)]

Всё очень плохо))

In [223]:
belgrad_locations = all_names(belgrad_df["text"], location_getter)

In [224]:
Counter(belgrad_locations).most_common(10)

[('белград', 234),
 ('сербия', 40),
 ('тот', 29),
 ('дунай', 28),
 ('сава', 26),
 ('царьград', 11),
 ('земуна', 10),
 ('восток', 9),
 ('великое', 9),
 ('европа', 9)]

Всё чуть получше))

### ПО ИМЕНАМ

Посмотрим на локации, которые употребляются рядом с Костой и Афанасием Свиларами. 

In [192]:
name_kosta = re.compile("Кост")
name_aphanas = re.compile("Афанас")
name_svilar = re.compile("Свилар")

In [193]:
print (aphanas)

[<aphanas>
Ведь каждый идиоритмик молчит сам по себе, а кенобиты хранят свою общую тишину. Одинокие люди возделывают молчание, словно пшеничное поле: вспахивают, открывают его пространство, углубляют борозды, поливают, чтобы зерна взошли, чтобы колосья вытянулись как можно выше, ибо только молчанием можно достичь Бога, но не криком, хоть ты надорвись кричамши… Напротив того, общинники не направляют лелеемую ими тишину навстречу Богу, но воздвигают ее, точно плотину, перед той частью мира, которая им не принадлежит и которой они хотят завладеть; они ограждают себя тишиною, защищаются ею или же насылают тишину на свою добычу, как охотничью собаку. Помня при этом, что охотничьи собаки бывают и хорошие, и плохие…
«Кругом одно горе, и все мы в нем точно рыба в воде», — думал неудавшийся архитектор Афанасий Свилар, ощущая сорок пятый год своей жизни, как запах чужого пота.
С 1950— го по 1956 год он учился в Белграде на архитектурном факультете. Тогда-то он и выяснил, что верхняя губа дана дл

In [194]:
aph = ""
for i in sent_tokenize(aphanas):
    if re.findall(name_aphanas, i) or re.findall(name_svilar, i):
        aph += i

In [202]:
location_getter(aph)

['мая',
 'быка',
 'гребенку',
 'Мркша',
 'одна пара',
 'Сава',
 'Дунай',
 'Никола',
 'Никола',
 'Белграда',
 'Дунае',
 'Никола',
 'Никола',
 'куда',
 'его саблю',
 'куда',
 'Никола',
 'Греции',
 'Албании',
 'Грецию',
 'того',
 'немецки',
 'куда',
 'того',
 'драхму',
 'драхму',
 'того',
 'того',
 'того',
 'нее',
 'Ивироне',
 'куда',
 'нее',
 'кармана',
 'того',
 'того',
 'кармане',
 'того',
 'Белграду',
 'Белград',
 'Никола',
 'Николы',
 'Белград']

In [205]:
kost = ""
for i in sent_tokenize(aphanas):
    if re.findall(name_kosta, i):
        kost += i

In [206]:
location_getter(kost)

['его саблю', 'куда', 'Албании', 'Грецию', 'кармане', 'Белград']

Результаты плохие. Попробуем "почти вручную", т.к. нам нужно все равно посмотреть на локации, связанные с Костой, для построения его маршрута.

### Коста

In [53]:
kosta = ""

with open("kosta.txt", "r+") as f:
    for line in f:
        if line.strip():
            kosta += line

In [62]:
soup = BeautifulSoup(kosta, 'html.parser')
kosta_ = str(soup.find_all("kosta"))
kosta_texts = kosta_.split("<kosta>")[1:]

In [88]:
kosta_df = pd.DataFrame({
        'text': kosta_texts,
        'tag': "kosta"
 })

In [98]:
kosta_df["bag"] = kosta_df["text"].apply(text_to_bag2)
kosta_df["sents"] = kosta_df["text"].apply(sent_tokenize)

In [99]:
def text_to_bag2(text):
    bagwords = text.split() 
    return bagwords

In [100]:
kosta_df

,text,tag,bag,sents,sents_bag
0,\nЕго преследовало одно воспоминание. Однажды ...,kosta,"[Его, преследовало, одно, воспоминание., Однаж...","[\nЕго преследовало одно воспоминание., Однажд...",NaN
1,"\nПредполагая, что майор Коста Свилар попал из...",kosta,"[Предполагая,, что, майор, Коста, Свилар, попа...","[\nПредполагая, что майор Коста Свилар попал и...",NaN
2,"\n— Стойте, стойте! — воскликнула, разволновав...",kosta,"[—, Стойте,, стойте!, —, воскликнула,, разволн...","[\n— Стойте, стойте!, — воскликнула, разволнов...",NaN
3,"\n— Это твой сад, — сказал ему монах, — вообще...",kosta,"[—, Это, твой, сад,, —, сказал, ему, монах,, —...","[\n— Это твой сад, — сказал ему монах, — вообщ...",NaN
4,\n— Все они для мира погибли и в пустыне не сп...,kosta,"[—, Все, они, для, мира, погибли, и, в, пустын...",[\n— Все они для мира погибли и в пустыне не с...,NaN


In [101]:
kosta_df["sents_bag"] = np.nan

In [102]:
for n, sents in enumerate(kosta_df["sents"]):
    lst = []
    for sent in sents:
        lst.append(text_to_bag2(sent)[1:])
    kosta_df["sents_bag"].iloc[n] = lst

In [103]:
kosta_df

,text,tag,bag,sents,sents_bag
0,\nЕго преследовало одно воспоминание. Однажды ...,kosta,"[Его, преследовало, одно, воспоминание., Однаж...","[\nЕго преследовало одно воспоминание., Однажд...","[[преследовало, одно, воспоминание.], [в, детс..."
1,"\nПредполагая, что майор Коста Свилар попал из...",kosta,"[Предполагая,, что, майор, Коста, Свилар, попа...","[\nПредполагая, что майор Коста Свилар попал и...","[[что, майор, Коста, Свилар, попал, из, Албани..."
2,"\n— Стойте, стойте! — воскликнула, разволновав...",kosta,"[—, Стойте,, стойте!, —, воскликнула,, разволн...","[\n— Стойте, стойте!, — воскликнула, разволнов...","[[Стойте,, стойте!], [воскликнула,, разволнова..."
3,"\n— Это твой сад, — сказал ему монах, — вообще...",kosta,"[—, Это, твой, сад,, —, сказал, ему, монах,, —...","[\n— Это твой сад, — сказал ему монах, — вообщ...","[[Это, твой, сад,, —, сказал, ему, монах,, —, ..."
4,\n— Все они для мира погибли и в пустыне не сп...,kosta,"[—, Все, они, для, мира, погибли, и, в, пустын...",[\n— Все они для мира погибли и в пустыне не с...,"[[Все, они, для, мира, погибли, и, в, пустыне,..."


In [105]:
names = []
for i, text in enumerate(kosta_df["sents_bag"]):
    for sent in text:
        for word in sent:
            if word[0].isupper():
                names.append(word)

In [107]:
print (names)

['Колодец', 'Коста', 'Свилар', 'Албании', 'Грецию', 'Афанасий', 'Свилар', 'Адриатики.', 'Баре', 'Корфу,', 'Итаки', 'Ионическим', 'Стойте,', 'Василия.', 'Вы', 'И', 'Они', 'Святой', 'Афонских', 'Это', 'Монах', 'Наверное,', 'Костой', 'Свиларом,', 'Все', 'Я', 'Свилар,', 'Свилар,', 'Святой', 'Святой', 'Свилар', 'Кто', 'Сербии.', 'Хилендар,', 'Введения', 'Богородицы,', 'Югославии,', 'Свилару', 'Свилар', 'Косту,', 'Что', 'Югославии']


In [108]:
len(names)

42

Теперь вытащим из этого списка локации и сделаем карту.

In [1]:
%%html
<img src = "https://pp.userapi.com/c848536/v848536818/1a6f2d/RLoPlBOynY0.jpg">